In [7]:
import numpy as np 
import scipy as sp  
import random     
import bokeh 
import csv
import pandas as pd 
import struct
import os
import pickle


In [8]:
network_dir = '20171109-162832'
# IMPORT THE PARAMS SOMEHOW.
#df = pd.read_csv('../data/20170427-stationary-2-leith-croydon.csv')

#Get the test idx's and any other post- training data necessary
with open(os.path.join(network_dir,'data.pkl'),'rb') as pkl_file:
    data_pkl = pickle.load(pkl_file)
    test_idxs = data_pkl['test_idxs']
    data_pool_filename = data_pkl['data_pool']
# HACK I have to import parameters from a child folder. Here I cmd(touch) __init__ so I can do an import.
fname = os.path.join(network_dir,"__init__.py")
try:
    os.utime(fname, None)
except OSError:
    open(fname, 'a').close()

# This is how you can import using a variable string.
import importlib
params = importlib.import_module(network_dir+".parameters")
parameters = params.parameters #Simplicity.

#TODO Also pkl the data_pool name.
# DO I also want to pkl the distance report?
# -- one feature at a time.


In [9]:
#Pull out a track, sort by time_idx. Is golden?
data_pkl


{'best_params': {'AAA': 'Maru reference',
  'RNN_cell': 'LSTMCell',
  'aug_function': <bound method Random.uniform of <random.Random object at 0x32ef2c0>>,
  'aug_range': (-3, 3),
  'augmentation_chance': 0.0,
  'batch_size': 100,
  'd_thresh_top_n': 1,
  'data_format': 'ibeo',
  'debug': False,
  'decrement_steps': 1000,
  'device': 'gpu:0',
  'dropout_prob': 0.5,
  'early_stop_cf': 30,
  'embedding_dropout': 0.0,
  'embedding_size': 512,
  'evaluation_metric_type': 'validation_loss',
  'feed_future_data': False,
  'first_loss_only': False,
  'hyper_learning_rate_args': (-3, 0),
  'hyper_learning_rate_fn': <bound method Random.uniform of <random.Random object at 0x32ef2c0>>,
  'hyper_reg_embedding_beta_args': None,
  'hyper_reg_embedding_beta_fn': <bound method Random.uniform of <random.Random object at 0x32ef2c0>>,
  'hyper_reg_l2_beta_args': (-16, -8),
  'hyper_reg_l2_beta_fn': <bound method Random.uniform of <random.Random object at 0x32ef2c0>>,
  'hyper_rnn_size_args': (128, 1025)

In [10]:
#Get data
master_pool = pd.read_pickle(os.path.join('..',os.path.join("data_pool",data_pool_filename)))

In [11]:
# Here  I want to run the network over the entire test set. 
master_pool.columns


Index([u'dest_1_hot', u'destination', u'destination_vec', u'origin',
       u'track_class', u'track_idx', u'encoder_sample', u'decoder_sample',
       u'distance', u'distance_to_exit', u'index', u'ObjectId',
       u'Classification', u'ObjectPredAge', u'Timestamp', u'AbsVelocity',
       u'ObjBoxOrientation', u'uniqueId', u'Object_X', u'Object_Y',
       u'track_time_idx'],
      dtype='object')

In [12]:
#master_pool[master_pool['track_idx']==1].sort_values('time_idx',ascending=True)

#Find the longest track for each track_class

'north-east'
longest_track_dict = {}
for track_class in master_pool['track_class'].unique():
    longest_track_dict[track_class] = master_pool[master_pool['track_class']==track_class]['track_idx'].mode()

In [13]:
single_track_pd = master_pool[master_pool['track_idx']==1]

In [14]:
single_track_pd
# ACTIONABLES:
    # Spawn a batchhandler using the test data
    # Run the network over the entire data set
    # Save back into THIS dataframe.
    # Then do lookups.


,dest_1_hot,destination,destination_vec,origin,track_class,track_idx,encoder_sample,decoder_sample,distance,distance_to_exit,...,ObjectId,Classification,ObjectPredAge,Timestamp,AbsVelocity,ObjBoxOrientation,uniqueId,Object_X,Object_Y,track_time_idx
0,"[0.0, 0.0, 1.0]",south,2,north,north-south,1,"[[-38.8615, -4.94367, 0.235566, 10.4011], [-38...","[[-28.7537, -4.77792, 0.00362321, 9.94607], [-...",-5.991319,-23.722221,...,7,5,0,1.496307e+09,9.824775,0.015429,3,-29.17840,-4.75653,0
0,"[0.0, 0.0, 1.0]",south,2,north,north-south,1,"[[-38.4356, -4.87701, 0.234533, 10.4404], [-38...","[[-28.3668, -4.68678, 0.035002, 9.85412], [-27...",-5.566080,-23.296983,...,7,5,0,1.496307e+09,9.946073,0.003623,3,-28.75370,-4.77792,1
0,"[0.0, 0.0, 1.0]",south,2,north,north-south,1,"[[-38.0059, -4.81193, 0.231392, 10.4845], [-37...","[[-27.9562, -4.70652, 0.0223083, 9.92408], [-2...",-5.168590,-22.899493,...,7,5,0,1.496307e+09,9.854118,0.035002,3,-28.36680,-4.68678,2
0,"[0.0, 0.0, 1.0]",south,2,north,north-south,1,"[[-37.5832, -4.7506, 0.227313, 10.5045], [-37....","[[-27.5734, -4.59931, 0.0584351, 9.8219], [-27...",-4.757516,-22.488419,...,7,5,0,1.496307e+09,9.924083,0.022308,3,-27.95620,-4.70652,3
0,"[0.0, 0.0, 1.0]",south,2,north,north-south,1,"[[-37.1997, -4.81661, 0.180036, 10.3543], [-36...","[[-27.1945, -4.53808, 0.072146, 9.76224], [-26...",-4.359987,-22.090889,...,7,5,0,1.496307e+09,9.821898,0.058435,3,-27.57340,-4.59931,4
0,"[0.0, 0.0, 1.0]",south,2,north,north-south,1,"[[-36.7624, -4.80087, 0.161098, 10.3946], [-36...","[[-26.8112, -4.43483, 0.102962, 9.69351], [-26...",-3.976171,-21.707074,...,7,5,0,1.496307e+09,9.762238,0.072146,3,-27.19450,-4.53808,5
0,"[0.0, 0.0, 1.0]",south,2,north,north-south,1,"[[-36.3193, -4.78286, 0.143312, 10.4479], [-35...","[[-26.4077, -4.42008, 0.0975049, 9.74548], [-2...",-3.579208,-21.310111,...,7,5,0,1.496307e+09,9.693505,0.102962,3,-26.81120,-4.43483,6
0,"[0.0, 0.0, 1.0]",south,2,north,north-south,1,"[[-35.8799, -4.76309, 0.12834, 10.4897], [-35....","[[-26.0294, -4.31243, 0.127592, 9.67077], [-25...",-3.175439,-20.906341,...,7,5,0,1.496307e+09,9.745478,0.097505,3,-26.40770,-4.42008,7
0,"[0.0, 0.0, 1.0]",south,2,north,north-south,1,"[[-35.4332, -4.77369, 0.10248, 10.534], [-34.9...","[[-25.5906, -4.35386, 0.0969164, 9.84907], [-2...",-2.782120,-20.513023,...,7,5,0,1.496307e+09,9.670775,0.127592,3,-26.02940,-4.31243,8
0,"[0.0, 0.0, 1.0]",south,2,north,north-south,1,"[[-34.992, -4.75042, 0.0903241, 10.5789], [-34...","[[-25.1357, -4.2814, 0.110828, 9.98495], [-24....",-2.341369,-20.072271,...,7,5,0,1.496307e+09,9.849073,0.096916,3,-25.59060,-4.35386,9


In [15]:

# COlors are no longer used to ID a track.
from matplotlib import pyplot as plt
Nlines = 260 
from itertools import permutations     
color_lvl = 8     
rgb = np.array(list(permutations(range(0,256,color_lvl),3))) 
rgb_m = np.array(list(permutations(range(0,256,color_lvl),3)))/255.0

from random import sample
colors = sample(rgb,Nlines) 
colors_m = sample(rgb_m,Nlines)

#struct.pack('BBB',*rgb).encode('hex')

#df['color'] = [tuple(colors[i]) for i in df.ObjectId]
df['color'] = ["#" + struct.pack('BBB',*colors[i]).encode('hex') for i in df.ObjectId]
df['color_m'] = [tuple(colors_m[i]) for i in df.ObjectId]

NameError: name 'df' is not defined

In [16]:

fill_color= [color if flag else None for color, flag in zip(df['color'],df['mobile'])]


NameError: name 'df' is not defined

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()

In [ ]:
from bokeh.plotting import figure
from bokeh.layouts import layout, widgetbox
from bokeh.models import ColumnDataSource, HoverTool, Div
from bokeh.models.widgets import Slider, Select, TextInput
from bokeh.io import curdoc, push_notebook
from ipywidgets import interact
import ipywidgets

output_notebook()

time_max = max(df['Timestamp'])
time_min = min(df['Timestamp'])

time_slider = Slider(start=time_min,end=time_max,value=time_min,step=1,title="Time")
source_rect = ColumnDataSource(data=dict(x=[], y=[], width=[], height=[],angle=[],RefPointLocation=[],
                                    color=[], title=[], year=[], revenue=[], alpha=[]))
source_circle = ColumnDataSource(data=dict(x=[], y=[], line_color=[], fill_color=[],RefPointLocation=[],
                                           title=[], year=[], revenue=[], alpha=[]))
source_crossH = ColumnDataSource(data=dict(x=[], y=[], width=[], height=[],angle=[],RefPointLocation=[],
                                    color=[], title=[], year=[], revenue=[], alpha=[]))
source_crossW = ColumnDataSource(data=dict(x=[], y=[], width=[], height=[],angle=[],RefPointLocation=[],
                                    color=[], title=[], year=[], revenue=[], alpha=[]))
source_ref_pt = ColumnDataSource(data=dict(x=[], y=[], line_color=[], fill_color=[], RefPointLocation=[],
                                           title=[], year=[], revenue=[], alpha=[]))
source_seg = ColumnDataSource(data=dict(x0=[], y0=[], x1=[], y1=[], line_color=[], fill_color=[], RefPointLocation=[],
                                           title=[], year=[], revenue=[], alpha=[]))
source_seg_accel = ColumnDataSource(data=dict(x0=[], y0=[], x1=[], y1=[], line_color=[], fill_color=[], RefPointLocation=[],
                                           title=[], year=[], revenue=[], alpha=[]))
#Sliders:
slider_time = ipywidgets.FloatSlider(
    value = (time_max+time_min)/2, min=time_min, max=time_max, step = 0.1, description="Time",slider_color="red")
window_slider = ipywidgets.FloatSlider(
    value = 5, min=1, max=time_max, step = 0.1, description="Time Window",slider_color="blue")
min_conf_slider = ipywidgets.FloatSlider(
    value = 5, min=0, max=100, step = 1, description="Min Classification Score",slider_color="green")


hover = HoverTool(tooltips=[
    ("ObjectId", "@ObjectId"),
    ("RefPointLocation", "@RefPointLocation"),
    #("$", "@revenue")
])

p = figure(plot_height=600, plot_width=700, title="", tools=[hover],x_range=(-40,20),y_range=(-60,40))
#p = figure(plot_height=600, plot_width=700, title="", tools=[hover],x_range=(-100,0),y_range=(-50,50))
#p.image_url(['intersection3.png'],-78.86,40.10,217.62,128.30)
p.rect(x="x", y="y", width="width",height="height", angle="angle",source=source_crossH, fill_color="color",
       line_color="color")
p.circle(x="x", y="y", source=source_circle, size=4, fill_color="fill_color", line_color="line_color")
p.circle(x="x", y="y", source=source_ref_pt, size=3, fill_color="black", line_color="black")
p.rect(x="x", y="y", width="width",height="height", angle="angle",source=source_rect, fill_color=None,
       line_color="color")
p.rect(x="x", y="y", width="width",height="height", angle="angle",source=source_crossW, fill_color="color",
       line_color="color")
p.segment(x0='x0',y0='y0',x1='x1',y1='y1',line_width=2,line_color='orange',source=source_seg)
p.segment(x0='x0',y0='y0',x1='x1',y1='y1',line_width=2,line_color='pink',source=source_seg_accel)


show(p,notebook_handle=True)

def select_data(time_val, time_window, min_conf):
    #time_val = time_slider.value
    #t_min = time_val-(float(time_window)/2.0)
    #t_max = time_val+(float(time_window)/2.0)
    t_min = time_val-time_window
    t_max = time_val

    selected = df[
        (df.Timestamp > t_min)
        &
        (df.Timestamp < t_max)
        &
        (df.Classification >3 )
        &
        (df.ClassCertainty > min_conf)
    ]
    
    return_df = pd.DataFrame()
    
    for obj_id in selected.ObjectId.unique():
        object_df = selected[selected.ObjectId==obj_id].sort_values(['Timestamp'],ascending=False)
        selected_record=object_df.iloc[0].copy()
        if len(object_df) > 1:
            selected_record['AbsAccel_X'] = object_df['AbsVelocity_X'].iloc[0] - object_df['AbsVelocity_X'].iloc[1]
            selected_record['AbsAccel_Y'] = object_df['AbsVelocity_Y'].iloc[0] - object_df['AbsVelocity_Y'].iloc[1]
        else:
            selected_record['AbsAccel_X'] = 0
            selected_record['AbsAccel_Y'] = 0
        return_df = return_df.append(selected_record)
        
    # Only draw box if car was seen in last 0.5 seconds. 
    # I really only want if it was seen at THIS timestep, but this is close enough
    recent_only_df = return_df[return_df['Timestamp']> (time_val-0.2)]
    return recent_only_df, selected #selected

def update(time_value=slider_time,time_window=window_slider,min_conf=min_conf_slider):
    df_rect, df_circle = select_data(time_value,time_window,min_conf)
    #print df_rect
    source_rect.data = dict(
        x=df_rect['ObjBoxCenter_X'],
        y=df_rect['ObjBoxCenter_Y'],
        width=df_rect['ObjBoxSize_X'],
        height=df_rect['ObjBoxSize_Y'],
        angle=df_rect['ObjBoxOrientation'],
        color=df_rect['color'],
        ObjectId=df_rect['ObjectId'],
        RefPointLocation = df_rect['RefPointLocation'],
        Timestamp=df_rect['Timestamp']
    )
    source_ref_pt.data = dict(
        x=df_rect['RefPointCoords_X'],
        y=df_rect['RefPointCoords_Y'],
        ObjectId=df_rect['ObjectId'],
        RefPointLocation = df_rect['RefPointLocation'],
        Timestamp=df_rect['Timestamp']
    )
    source_seg.data = dict(
        x0=df_rect['ObjBoxCenter_X'],
        y0=df_rect['ObjBoxCenter_Y'],
        x1=df_rect['ObjBoxCenter_X'].add(df_rect['AbsVelocity_X']),
        y1=df_rect['ObjBoxCenter_Y'].add(df_rect['AbsVelocity_Y']),
        ObjectId=df_rect['ObjectId'],
        RefPointLocation = df_rect['RefPointLocation'],
        Timestamp=df_rect['Timestamp']
    )
    source_seg_accel.data = dict(
        x0=df_rect['ObjBoxCenter_X'],
        y0=df_rect['ObjBoxCenter_Y'],
        x1=df_rect['ObjBoxCenter_X'].add(10*df_rect['AbsAccel_X']),
        y1=df_rect['ObjBoxCenter_Y'].add(10*df_rect['AbsAccel_Y']),
        ObjectId=df_rect['ObjectId'],
        RefPointLocation = df_rect['RefPointLocation'],
        Timestamp=df_rect['Timestamp']
    )
    source_circle.data = dict(
        x=df_circle['ObjBoxCenter_X'],
        y=df_circle['ObjBoxCenter_Y'],
        fill_color= [color if not flag else "#FFFFFF" for color,flag in zip(df_circle['color'],df_circle['ObjPrediction'])],
        line_color=df_circle['color'],
        ObjectId=df_circle['ObjectId'],
        #RefPointLocation = df_rect['RefPointLocation'],
        Timestamp=df_circle['Timestamp']
    )
    source_crossW.data = dict(
        x=df_rect['ObjBoxCenter_X'],
        y=df_rect['ObjBoxCenter_Y'],
        width=df_rect['ObjBoxCenterSigma_X'],
        height=[0.1]*len(df_rect),
        angle=df_rect['ObjBoxOrientation'],
        color=df_rect['color'],
        ObjectId=df_rect['ObjectId'],
        RefPointLocation = df_rect['RefPointLocation'],
        Timestamp=df_rect['Timestamp']
    )
    #'ObjBoxCenterSigma_Y'
    source_crossH.data = dict(
        x=df_rect['ObjBoxCenter_X'],
        y=df_rect['ObjBoxCenter_Y'],
        width=[0.1]*len(df_rect),
        height=df_rect['ObjBoxCenterSigma_Y'],
        angle=df_rect['ObjBoxOrientation'],
        color=df_rect['color'],
        ObjectId=df_rect['ObjectId'],
        RefPointLocation = df_rect['RefPointLocation'],
        Timestamp=df_rect['Timestamp']
    )
    
    push_notebook()


update(time_max,1,5)  # initial load of the data



In [ ]:
#If this fails run $ jupyter nbextension enable --py --sys-prefix widgetsnbextension

interact(update,time_val=slider_time,time_window=window_slider,min_conf=min_conf_slider)
